In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API")

In [5]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.conditions import TextMentionTermination

In [4]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key,
)

In [ ]:
planner_prompt = """
You are a travel planner. 
You will be given a user query and you need to plan a trip for the user.
Yoo should create a plan of the trip that will be passed to the other agents to find information in the web.
Give clear instructions what information should be googled.
"""

search_prompt = """
You are a search agent.
You will be given a query and you need to find the information in the web.
You should use the Google search engine to find the information.
"""

critic_prompt = """
You are a critic agent.
You will be given a plan and you need to critique it.
You should provide constructive feedback on the plan.
Respond with 'APPROVE' to when user is satisfied.
"""

search = GoogleSearchAPIWrapper(k=10)

web_tool = LangChainToolAdapter(
    Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run,
    )
)

planner_agent = AssistantAgent(
    "planner",
    model_client=model_client,
    system_message=planner_prompt,
)

search_agent = AssistantAgent(
    "search",
    model_client=model_client,
    system_message=search_prompt,
    tools=[web_tool],
)

critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message=critic_prompt,
)

termination = TextMentionTermination("APPROVE")

user_proxy = UserProxyAgent("user_proxy", input_func=input)

team = MagenticOneGroupChat(
    [planner_agent, search_agent, critic_agent, user_proxy], 
    termination_condition=termination
)

In [10]:
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."



model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key,
)

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

await Console(agent.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_cFdGv9fSCQoWELhgeqSgztwY', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_cFdGv9fSCQoWELhgeqSgztwY', is_error=False)]
---------- weather_agent ----------
The current weather in New York is 73 degrees and sunny.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_cFdGv9fSCQoWELhgeqSgztwY', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_cFdGv9fSCQoWELhgeqSgztwY', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content='The current weather in New York is 73 degrees and sunny.', type='TextMessage')], stop_reason=None)